# Project 3: Web APIs & NLP Part 2 Data Cleaning

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
# import raw data of initial scrapes of techsupport subreddit
techsupport = pd.read_csv('data2/raw_techsupport_initial_scrape.csv')
# import raw data of initial scrapes of creative writing subreddit
creative = pd.read_csv('data2/raw_creative_initial_scrape.csv')


In [3]:
# to make DataFrames easier to navigate through during cleaning/EDA
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)



## Cleaning Data

In [4]:
# checking shape of techsupport subreddit DataFrame
techsupport.shape

(20040, 94)

In [5]:
# checking shape of creative writing subreddit DataFrame
creative.shape

(20095, 90)

In [6]:
# looking at columns to decide which ones to drop
techsupport.columns

Index(['Unnamed: 0', 'all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_richtext', 'link_flair_text_color', 'link_flair_type',
       'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts',
       'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'removed_by_category', 'retrieved_on', 'score', 'selftext',
       'send_replies', 'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'subreddit_type', 'th

In [7]:
# selecting 'selftext','author' and 'title' columns as these seem most relevant
techsupport = techsupport[['selftext', 'author','title']]

In [8]:
creative = creative[['selftext', 'author','title']]

In [9]:
techsupport.head()

,selftext,author,title
0,[removed],DihedralStem,Unexpected global perspective
1,"During college I worked for the University helpdesk. I had just gotten my first promotion and was finally allowed to go on-site and work in our walk-in area. One of the people working phones got a call from a student about their Nintendo Switch not connecting to the Residence Hall internet. This is a somewhat common call as Switches are incompatible with the 802.1X authentication our network used. \n\nThe person working the phone did their best to explain this in English to an astonished customer, and long story short the customer flipped. He threatened the phone agent, found our address, then said he'd be over in 10 minutes to kill us all unless we let his Switch on the network. Essentially being a glorified receptionist this was relayed to me and fulltime staff were made aware and decided to invite the University Police over, who happened to be our office neighbors. \n\n10 minutes go by and there's me, 3 staff members, and 2 cops standing in our dingy little walk-up area, when a student who must've been 5'6"" 120 lbs walked in with one hand in a fist and the other cradling his Switch. Beyond that, it wasn't particularly eventful but it was the first arrest of several I saw in my two years working there.",FrostyPaddy,My First Helpdesk Arrest
2,"&gt; $worksTooMuch: Hey, Ging, can you look at this item in the call queue for me?\n\n&gt; $gingBeard: Yeah. Uh, does that say they've been on hold for six hours?\n\n&gt; $worksTooMuch: OK. I wanted to make sure I wasn't misreading that. I'm going to change the phone priority and have $breaker take it. Would you warn him?\n\n&gt; $gingBeard: $Breaker? If you say so.\n\nThe MSP we worked for would gave discounts to customers (all businesses) if they agreed to be bumped down the call priority. I guess this was a fine enough deal for companies that only had us for calls outside business hours. Then, there's was $KioskCo, who sent us tickets twice in the year I was there. Both handled by $breaker and both involved a several hour wait because they decided they wanted max discount and were always at the bottom of the call priority. Always. \n\nAs $Breaker told it, the call went like this:\n\n*Introductions*\n\n&gt; $KioskCo: Because of [ERROR: REASON NOT FOUND] the last girl started a print job that's going to print, uh, maybe six or seven thousand pages? Well the printer started smoking so I called you. \n\n&gt; $Breaker: The printer is running a six or seven thousand page job and started smoking when you called six hours ago?\n\n&gt; $KioskCo: Yes.\n\n&gt; $Breaker: Is, is the printer still running? \n\n&gt; $KioskCo: Yes. The print job isn't finished. And the smoke coming out of it is getting pretty dark\n\nAtleast 30 seconds of uninterrupted silence pass as his prefrontal cortex BSoD's and reboots. \n\n&gt; $Breaker: Can you stop the print job? And then unplug the printer.\n\n&gt; $KioskCo: But the print job isn't finished. \n\n&gt; $Breaker: I'll, I'll fix that after. \n\nAfter convincing the woman to unplug the printer he got her to take the printed pages out of the room and call (I think) an electrician and their local non-emergency emergency services phone number.\n\nI don't know if he ever fixed the interrupted print job.",WantDebianThanks,LP0: Printer on Fire
3,"So as I mentioned in my previous post I'm a call center agent for a bank and our training recently finished last week and so I'll now have full shifts starting earlier today,the problem with working in the Philippines and your clientele being in the US is that the time zones are wonky,seeing as I'm day shift 6am-3pm AKA 5pm to 2am EST \n\nNow one may think hey at least it'll slow down that last few hours right,that's what I thought as well,but being the only 3 people there in production as well as my coworker who shares a shift with me and my support,means that we're the only one taking calls and so the calls come one after the other,or queuein

In [10]:
# checking for null values in selftext column
techsupport['selftext'].isna().sum()

2669

In [11]:
# replacing removed/deleted textposts with null values so that we can drop them
techsupport['selftext']=techsupport['selftext'].replace('[removed]', np.nan)
techsupport['selftext']=techsupport['selftext'].replace('[deleted]', np.nan)
techsupport=techsupport.dropna(subset=['selftext'])
creative['selftext']=creative['selftext'].replace('[removed]', np.nan)
creative['selftext']=creative['selftext'].replace('[deleted]', np.nan)
creative=creative.dropna(subset=['selftext'])

In [12]:
# checking shape of  DataFrame and number of null values after deleting
print(f"The shape of the tech support dataframe is {techsupport.shape} and there are {techsupport['selftext'].isna().sum()} null values")
print(f"The shape of the creative writing dataframe is {creative.shape} and there are {creative['selftext'].isna().sum()} null values")


The shape of the tech support dataframe is (14004, 3) and there are 0 null values
The shape of the creative writing dataframe is (18096, 3) and there are 0 null values


In [13]:
# dropping duplicate values in selftext and choosing to keep the first
techsupport=techsupport.drop_duplicates(subset='selftext', keep='first', inplace=False)
creative=creative.drop_duplicates(subset='selftext', keep='first', inplace=False)

In [14]:
# creating dictionary of subreddit moderators so that we can filter out posts made by them
techsupport_mods = ['Kumorigoe','Willz12h','doctoroctoclops','mandevwin','PipeItToDevNull','Jalad25','ocupi','BigCatSanctuary','Geeknificient','manawolf146']
creative_mods = ['KingBeetle','carbon107','KrisAlmighty','metastasis_d','NO_TOUCHING__lol','JesperTV']
techsupport = techsupport[~techsupport['author'].isin(techsupport_mods)]
creative = creative[~creative['author'].isin(creative_mods)]

In [15]:
# checking shape of  DataFrame after filtering out moderator posts
print(f"The shape of the tech support dataframe is {techsupport.shape}")
print(f"The shape of the creative writing dataframe is {creative.shape}")


The shape of the tech support dataframe is (5134, 3)
The shape of the creative writing dataframe is (7150, 3)


In [16]:
# thru the magic of Ben Mathis and their regex abilities I am able filter out any URLs
techsupport['selftext']=techsupport['selftext'].apply(lambda x: re.sub(r'http\S+', '', x))
creative['selftext']=creative['selftext'].apply(lambda x: re.sub(r'http\S+', '', x))

In [17]:
# replacing titles that are just 'title' with null values so we can remove them
techsupport['title']=techsupport['title'].replace('title', np.nan)
creative['title']=creative['title'].replace('title', np.nan)
creative=creative.dropna(subset=['title'])
techsupport=techsupport.dropna(subset=['title'])

In [18]:
# dropping duplicate titles in title column
techsupport=techsupport.drop_duplicates(subset='title', keep='first', inplace=False)
creative=creative.drop_duplicates(subset='title', keep='first', inplace=False)

In [19]:
# dropping blank entries in DataFrames
creative = creative[~(creative['selftext'] =='')]
techsupport = techsupport[~(techsupport['selftext']=='')]
creative = creative[~(creative['title'] =='')]
techsupport = techsupport[~(techsupport['title']=='')]
creative = creative[~(creative['author'] =='')]
techsupport = techsupport[~(techsupport['author']=='')]

In [20]:
# checking cleaned DataFrames
print(f"The final shape of the cleaned tech support dataframe is {techsupport.shape[0]} rows and {techsupport.shape[1]} columns and there are {techsupport.isna().sum().sum()} null values")
print(f"The final shape of the cleaned creative writing dataframe is {creative.shape[0]} rows and {creative.shape[1]} columns and there are {creative.isna().sum().sum()} null values")


The final shape of the cleaned tech support dataframe is 5120 rows and 3 columns and there are 0 null values
The final shape of the cleaned creative writing dataframe is 6928 rows and 3 columns and there are 0 null values


In [22]:
# exporting cleaned dataframes of tech support and creative writing 
techsupport.to_csv('data2/cleaned_techsupport.csv', index=False)
creative.to_csv('data2/cleaned_creativewriting.csv',index=False)
